In [ ]:
#import relevant libraries 
import pandas as pd
import plotly.express as px
import panel as pn
import hvplot.pandas
import requests
import json
import numpy as np
import matplotlib.pyplot as plt
import json
from pathlib import Path
import alpaca_trade_api as tradeapi
from dotenv import load_dotenv
import os
from datetime import datetime, timedelta,date
from pandas import DataFrame

In [ ]:
## Get Market Data for S&P500 
#engage API keys by activating .env file for Alpaca Api
load_dotenv()
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")
api = tradeapi.REST(alpaca_api_key, alpaca_secret_key, api_version='v2')

#function to read the api data for stock ticker
#returns dataframe of closing price and daily returns for a given ticker symbol
def get_ticker_data(api,ticker):
    #load in historical data for provided ticker
    stock_data_df = api.alpha_vantage.historic_quotes(ticker, adjusted=True, output_format='pandas')

    #Clean Data
    

    #Sort earliest to latest. so that .pct_change() function works right.
    stock_data_df.sort_index(inplace=True, ascending=True)

    # Drop nulls
    stock_data_df.dropna(inplace=True)

    # drop duplicates
    stock_data_df.drop_duplicates(inplace=True)

    #count nulls 
    stock_data_df.isnull().sum()

    #create a dataframe column for the daily returns (pct_change) values and concat 
    returns_df = stock_data_df['5. adjusted close'].pct_change()
    stock_data_df = pd.concat([stock_data_df, returns_df], axis="columns", join="inner")

    #Change column names to avoid confusion
    columns = ['Open','High','Low','Close','Adjusted Close','Volume','Dividend Amount','Split Coefficient','Daily Returns']
    stock_data_df.columns = columns

    # Drop nulls
    stock_data_df.dropna(inplace=True)

    #drop duplicates
    stock_data_df.drop_duplicates(inplace=True) 
    return stock_data_df

In [ ]:
# function to get the SMA of the returns
# return EMA signal values dataframe
def get_EMA(stock_data_df,ticker,short_window,long_window): #typical short is 50 and long is 100
    # Grab just the `date` and `close` from the dataset
    ema_signals = stock_data_df.loc[:, ['Close']].copy()

    # Generate the short and long exponential moving averages (50 and 100 days, respectively)
    ema_signals["EWM50"] = ema_signals['Close'].ewm(span=short_window).mean()
    ema_signals["EWM100"] = ema_signals['Close'].ewm(span=long_window).mean()
    ema_signals["Signal"] = 0.0

    # Generate the trading signal 0 or 1,
    # where 0 is when the EWM50 is under the EWM100, and
    # where 1 is when the EWM50 is higher (or crosses over) the SMA100
    ema_signals["Signal"][short_window:] = np.where(
    ema_signals["EWM50"][short_window:] > ema_signals["EWM100"][short_window:], 1.0, 0.0)
    # Calculate the points in time at which a position should be taken, 1 or -1
    ema_signals["Entry/Exit"] = ema_signals["Signal"].diff()
    return ema_signals

In [ ]:
# function to get the Bolinger Bands of the returns
# return Bolinger Band signal values dataframe
def get_Bollinger(stock_data_df,ticker,bollinger_window, bollinger_window_long,no_of_std):
    # Grab just the `date` and `close` from the dataset
    bollinger_signals = stock_data_df.loc[:, ['Close']].copy()

    # Calculate rolling mean and standard deviation
    bollinger_signals['Bollinger middle'] = bollinger_signals['Close'].rolling(window=bollinger_window).mean()
    bollinger_signals['Bollinger Long']= bollinger_signals['Close'].rolling(window= bollinger_window_long).mean()
    bollinger_signals['Bollinger STD'] = bollinger_signals['Close'].rolling(bollinger_window).std()

    # Calculate upper and lowers bands of bollinger band
    bollinger_signals['Bollinger Upper']  = bollinger_signals['Bollinger middle'] + (bollinger_signals['Bollinger STD'] * no_of_std)
    bollinger_signals['Bollinger Lower']  = sbollinger_signals['Bollinger middle'] - (bollinger_signals['Bollinger STD'] * no_of_std)

    #Calculate Signals
    #bollinger_signals['Bollinger Long'] = np.where(bollinger_signals['Close'] < bollinger_signals['Bollinger Lower'], 1.0, 0.0)
    #bollinger_signals['Bollinger Short'] = np.where(bollinger_signals['Close'] > bollinger_signals['Bollinger Upper'], -1.0, 0.0)
    #bollinger_signals['Bollinger Signal'] = bollinger_signals['Bollinger Long'] + bollinger_signals['Bollinger Short']

    bollinger_signals['Signal'] = np.where(bollinger_signals['Close']<
    bollinger_signals['Bollinger Lower'], 1.0, 0.0)
    # Calculate the points in time at which a position should be taken, 1 or -1
    bollinger_signals["Entry/Exit"] = bollinger_signals["Signal"].diff()

    #btc_df.head()

    return 

In [ ]:
# function to get the MACD Value of the returns
# return MACD signal values dataframe
def get_MACD(stock_data_df,ticker, span1, span2,span3): ##typically span = 12, 26, 9
    # Grab just the `date` and `close` from the dataset
    macd_signals = stock_data_df.loc[:, ['Close']].copy()

    
    exp1 = macd_signals.Close.ewm(span1, adjust=False).mean()
    exp2 = macd_signals.Close.ewm(span2, adjust=False).mean()
    macd = exp1-exp2
    macd_out = macd.ewm(span3, adjust=False).mean()
    
    macd_signals["Signal"] = np.where(macd > macd_out, 1.0, 0.0)
    macd_signals["MACD Entry/Exit"] = mac_signals["Signal"].diff()

    return macd_signals


In [ ]:
#function to get all trading signals
#return data frame holding trading signals
def get_trading_signals(ema_signal_df,boligner_signals_df,macd_signals_df):
    trading_signals_df=pd.concat([ema_signal_df, bolinger_signal_df,mcad_signal_df],axis=1,    join="inner")
    trading_signals_df['Signal']=trading_signals_df['EMA Signal']+ trading_signals_df['Bolinger Signal']+ trading_signals_df['MACD Signal']
    trading_signals_df['Overall Entry/Exit']=trading_signals_df['Signal'].diff()
    return trading_signals_df

In [ ]:

#Plot Signals and Indicators Against Price
def plot_signals(trading_signals_df, ema_signals, mac_signals)
    # Visualize exit position relative to close price
    exit = trading_signals_df['Overall Entry/Exit'] == -1.0].hvplot.scatter(
        color='red',
        marker='v',
        size=200,
        legend=False,
        ylabel='Price in $',
        width=1000,
        height=400
    )

    # Visualize entry position relative to close price
    entry = trading_signals_df['Overall Entry/Exit'] == 1.0].hvplot.scatter(
        color='green',
        marker='^',
        size=200,
        legend=False,
        ylabel='Price in $',
        width=1000,
        height=400
    )

# Visualize close price for the investment
security_close = trading_signals_df[[f"{ticker}S&P 500 close"]].hvplot(
    line_color='lightgray',
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize moving averages
moving_avgs = ema_signals[['EWM50', 'EWM100']].hvplot(
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize bolinger bands
#bolinger_bands = Bolinger_signal_df[['EWM50', 'EWM100']].hvplot(
#    ylabel='Price in $',
#    width=1000,
#    height=400
#)

# Visualize moving averages
#macd = macd_signals[['EWM50', 'EWM100']].hvplot(
#    ylabel='Price in $',
#    width=1000,
#    height=400
#)

# Overlay plots
stock_signals_plot = security_close * moving_avgs * entry * exit
stock_signals_plot.opts(xaxis=None, title =f"Entry/Exit {ticker} Signals")